In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask
from shapely.geometry import Point
import plotly.graph_objects as go
import os
import math
from tqdm import tqdm
from shapely.geometry import LineString
from concurrent.futures import ProcessPoolExecutor, as_completed
import glob

In [ ]:
pip install whitebox

In [ ]:
from whitebox import WhiteboxTools
wbt = WhiteboxTools()
wbt.help()


# TERRIAN ANALYSIS

## FILL DEM

In [ ]:
wbt.fill_depressions(
    dem=r"file_path\output_AW3D30.tif", 
    output=r"file_path\Fill.tif", 
    fix_flats=True, 
    flat_increment=None, 
    max_depth=None, 
)


## FLOW DIRECTION 'D8_METHOD'

In [ ]:
wbt.d8_pointer(
    dem=r"file_path\Fill.tif", 
    output=r"file_path\D8.tif", 
    esri_pntr=False, 
)


## FLOW ACCUMULARION

In [ ]:
wbt.d8_flow_accumulation(
    i=r"file_path\D8.tif", 
    output=r"file_path\Fac.tif", 
    out_type="cells", 
    log=False, 
    clip=False, 
    pntr=True, 
    esri_pntr=False, 
)


## STREAMS

In [ ]:
wbt.extract_streams(
     flow_accum=r"file_path\Fac.tif", 
    output=r"file_path\Str.tif", 
    threshold=16000, # NO OF CELLS
    zero_background=False 
)


## Sanps 'opptional or U can Use the QGIS GUI'

### Snap Pour Points

In [ ]:
wbt.snap_pour_points(
    pour_pts=r"file_path\Snap.shp", 
    flow_accum=r"file_path\Fac.tif", 
    output=r"file_path\SPP.shp", 
    snap_dist=100, 
)


### jsnap

In [ ]:
wbt.jenson_snap_pour_points(
    pour_pts=r"file_path\SNAP.shp", 
    streams=r"file_path\Str.tif", 
    output=r"file_path\Jsnap", 
    snap_dist=200, 
)


## Watersheds

In [ ]:
wbt.watershed(
     d8_pntr=r'file_path\d8.tif', 
    pour_pts=r'file_path\SNAP.shp', 
    output=r'file_path\WS.TIFF',  
)


In [ ]:
wbt.raster_to_vector_polygons(
    i=r"file_path\WS.tifF", 
    output=r"file_path\WS.shp", 
)


## Streams

In [ ]:
wbt.raster_streams_to_vector(
    streams=r'file_path\Str.tif', 
    d8_pntr=r'file_path\D8.tif', 
    output=r'file_path\streams', 
    esri_pntr=False, 
)


## LFP

In [ ]:
wbt.longest_flowpath(
    dem=r"file_path\Fill.tif", 
    basins=r"file_path\WS.tiff", 
    output=r"file_path\LFP.shp" 
)


# LFP PARAMETERS


In [ ]:
# PARAMETERS
# -------------------------
shapefile_path\ = "LFP.shp"
dem_path = "PRO.tif"
ID_FIELD = "FID"      # 🔴 CHANGE THIS to your existing ID field name
output_gpkg = "LFP_with_metrics.gpkg"

# -------------------------
# LOAD DATA
# -------------------------
lines = gpd.read_file(shapefile_path\)

dem = rasterio.open(dem_path)
dem_array = dem.read(1)
dem_transform = dem.transform
dem_nrows, dem_ncols = dem_array.shape

# -------------------------
# HELPER FUNCTIONS
# -------------------------
def coords_to_index(xs, ys, transform):
    """Convert x, y coordinates to DEM row/col indices (vectorized)."""
    cols = ((xs - transform.c) / transform.a).astype(int)
    rows = ((ys - transform.f) / transform.e).astype(int)

    rows = np.clip(rows, 0, dem_nrows - 1)
    cols = np.clip(cols, 0, dem_ncols - 1)
    return rows, cols

def get_line_elevations(line_geom, dem_array, transform):
    """Extract elevations at line vertices."""
    coords = np.array(line_geom.coords)
    xs, ys = coords[:, 0], coords[:, 1]
    rows, cols = coords_to_index(xs, ys, transform)
    elevations = dem_array[rows, cols]
    return elevations[~np.isnan(elevations)]

def compute_tc(Lseg, S):
    """Time of concentration formula."""
    if Lseg <= 0 or S <= 0:
        return np.nan
    return 0.0195 * (Lseg ** 0.77) * (S ** -0.385)

def compute_metrics(row):
    geom = row.geometry
    elevations = get_line_elevations(geom, dem_array, dem_transform)

    if len(elevations) == 0:
        return None

    # -------------------------
    # BASIC METRICS
    # -------------------------
    L_total = geom.length
    elev_us = elevations[0]
    elev_ds = elevations[-1]
    elev_10 = np.percentile(elevations, 10)
    elev_85 = np.percentile(elevations, 85)

    # -------------------------
    # SEGMENT LENGTHS
    # -------------------------
    L1 = 0.15 * L_total
    L2 = 0.70 * L_total
    L3 = 0.15 * L_total

    # -------------------------
    # SLOPES
    # -------------------------
    slope1 = (elev_us - elev_85) / L1 if L1 > 0 else np.nan
    slope2 = (elev_85 - elev_10) / L2 if L2 > 0 else np.nan
    slope3 = (elev_10 - elev_ds) / L3 if L3 > 0 else np.nan

    # -------------------------
    # TIME OF CONCENTRATION
    # -------------------------
    Tc1 = compute_tc(L1, slope1)
    Tc2 = compute_tc(L2, slope2)
    Tc3 = compute_tc(L3, slope3)

    return {
        ID_FIELD: row[ID_FIELD],
        "length_m": L_total,
        "elev_up": elev_us,
        "elev_down": elev_ds,
        "elev_10": elev_10,
        "elev_85": elev_85,
        "slope_u15": slope1,
        "slope_m70": slope2,
        "slope_l15": slope3,
        "Tc_u15_min": Tc1,
        "Tc_m70_min": Tc2,
        "Tc_l15_min": Tc3,
        "Tc_total_min": Tc1 + Tc2 + Tc3
    }

# -------------------------
# PROCESS ALL LINES
# -------------------------
results = []

print(f"Processing {len(lines)} flow paths...")
for _, row in tqdm(lines.iterrows(), total=len(lines), desc="Flow paths"):
    metrics = compute_metrics(row)
    if metrics is not None:
        results.append(metrics)

df = pd.DataFrame(results)

# -------------------------
# ENSURE ID TYPE MATCH
# -------------------------
lines[ID_FIELD] = lines[ID_FIELD].astype(df[ID_FIELD].dtype)

# -------------------------
# JOIN RESULTS BACK
# -------------------------
lines_joined = lines.merge(
    df,
    on=ID_FIELD,
    how="left"
)

# -------------------------
# SAVE OUTPUT (BEST FORMAT)
# -------------------------
lines_joined.to_file(
    output_gpkg,
    layer="flow_paths",
    driver="GPKG"
)

print("✅ Processing complete")
print(f"✅ Output saved to: {output_gpkg}")

# join by nearest 

In [ ]:
# USER INPUTS
# ----------------------------------
watershed_fp = r"file_path\WS.shp"

lfp_gpkg = r"file_path\LFP_with_metrics.gpkg"
lfp_layer = "flow_paths"

output_fp = r"file_path\watersheds.shp"

target_epsg = 32636   # UTM zone example

# ----------------------------------
# LOAD DATA
# ----------------------------------
watersheds = gpd.read_file(watershed_fp)
flowpaths  = gpd.read_file(lfp_gpkg, layer=lfp_layer)

print("✔ Data loaded")

# ----------------------------------
# CRS CHECK & REPROJECT
# ----------------------------------
if watersheds.crs is None or flowpaths.crs is None:
    raise ValueError("❌ One or both layers have no CRS")

watersheds = watersheds.to_crs(epsg=target_epsg)
flowpaths  = flowpaths.to_crs(epsg=target_epsg)

print("✔ CRS aligned:", watersheds.crs)

# ----------------------------------
# MANUAL ONE-TO-ONE NEAREST JOIN
# ----------------------------------
nearest_idx = []

for ws in watersheds.geometry:
    dists = flowpaths.geometry.distance(ws)
    nearest_idx.append(dists.idxmin())

nearest_flowpaths = flowpaths.loc[nearest_idx].reset_index(drop=True)

# Distance column
nearest_flowpaths["dist_to_ws_m"] = [
    flowpaths.loc[idx].geometry.distance(ws)
    for idx, ws in zip(nearest_idx, watersheds.geometry)
]

# ----------------------------------
# MERGE ATTRIBUTES
# ----------------------------------
flow_attrs = nearest_flowpaths.drop(columns="geometry")

watersheds = pd.concat(
    [watersheds.reset_index(drop=True),
     flow_attrs.reset_index(drop=True)],
    axis=1
)

# ----------------------------------
# DELETE EXISTING SHAPEFILE (IF EXISTS)
# ----------------------------------
if os.path.exists(output_fp):
    base = os.path.splitext(output_fp)[0]
    for f in glob.glob(base + ".*"):
        os.remove(f)
    print("✔ Existing Shapefile removed")

# ----------------------------------
# SAVE OUTPUT (UPDATED SHP)
# ----------------------------------
os.makedirs(os.path.dirname(output_fp), exist_ok=True)
watersheds.to_file(output_fp)

print("✔ Output saved to:", output_fp)
print("✔ All watersheds updated with exactly one nearest flow path")

## Attributes And Export

In [ ]:
# INPUT FILE (overwrite same shapefile)
# ----------------------------------
input_fp = r"file_path\watersheds.shp"
output_fp = input_fp  # overwrite original file
excel_fp = r"file_path\watersheds_attributes.xlsx"  # Excel export path

# ----------------------------------
# LOAD SHAPEFILE
# ----------------------------------
gdf = gpd.read_file(input_fp)
print("✔ Shapefile loaded:", gdf.shape)

# ----------------------------------
# DROP COLUMNS 2 TO 7 AND LAST COLUMN (except geometry)
# ----------------------------------
geom_col = gdf.geometry.name
cols = list(gdf.columns)

# Drop columns 2 to 7 (index 1 to 6) if enough exist
if len(cols) > 7:
    drop_cols = cols[1:7]
    gdf = gdf.drop(columns=drop_cols)

# Drop last column if it's not geometry
if gdf.columns[-1] != geom_col:
    gdf = gdf.drop(columns=gdf.columns[-1])

# ----------------------------------
# ADD WATERSHED_NAME FIELD AFTER FID
# ----------------------------------
if 'FID' in gdf.columns:
    fid_index = gdf.columns.get_loc('FID')
else:
    fid_index = 0

n = len(gdf)
watershed_names = [f"Watershed_{i+1:02d}" for i in range(n)]
gdf.insert(fid_index + 1, 'Watershed_Name', watershed_names)

# ----------------------------------
# REORDER COLUMNS: FID → Watershed_Name → others → geometry
# ----------------------------------
other_cols = [c for c in gdf.columns if c not in ['FID', 'Watershed_Name', geom_col]]
gdf = gdf[['FID', 'Watershed_Name'] + other_cols + [geom_col]]
print("✔ Columns adjusted:", gdf.columns.tolist())

# ----------------------------------
# ROUND NUMERIC COLUMNS TO 2 DECIMALS (skip certain columns)
# ----------------------------------
# Columns to exclude from rounding
exclude_cols = ['FID', 'Watershed_Name',"slope_u15","slope_m70","slope_l15"]

# Select numeric columns
numeric_cols = gdf.select_dtypes(include=['float64', 'int64']).columns

# Keep only numeric columns not in exclude list
cols_to_round = [col for col in numeric_cols if col not in exclude_cols]

# Round them
for col in cols_to_round:
    gdf[col] = (gdf[col] * 100).round() / 100

print("✔ Numeric columns rounded to 2 decimals (excluded FID and Watershed_Name)")


# ----------------------------------
# SAVE CLEANED SHAPEFILE (overwrite)
# ----------------------------------
gdf.to_file(output_fp)
print("✔ Shapefile updated:", output_fp)

# ----------------------------------
# EXPORT ATTRIBUTE TABLE TO EXCEL (overwrite, remove last column)
# ----------------------------------
attr_table = gdf.drop(columns="geometry")
attr_table = attr_table.iloc[:, :-1]  # remove last column
attr_table.to_excel(excel_fp, index=False)
print("✔ Attribute table exported to Excel (last column removed, file overwritten)")


## PLOTTING

In [ ]:
from whitebox_workflows import WbPalette

palettes = [p for p in dir(WbPalette) if not p.startswith('_')]
print(palettes)


In [ ]:
# Read in the files
dem = wbe.read_raster(r'file_path\mexar.tif')
streams = wbe.read_vector(r'file_path\clipped_streams.shp')
LFP = wbe.read_vector(r'file_path\LFP.shp')
watershed = wbe.read_vector(r'file_path\WS.shp')
outlet = wbe.read_vector(r'file_path\outlet.shp')
study_area = wbe.read_vector(r'file_path\BASEMAP.shp')  # study area

# Create hillshade from DEM
hillshade = wbe.hillshade(dem)

# Create the figure
fig, ax = plt.subplots(figsize=(12,8))

# Show DEM
ax = show(dem, ax=ax, title='MEXAR', cmap=WbPalette.BlueGreenYellow, clip_percent=0.0, skip=2, colorbar_kwargs={'label': 'Elevation (m)', 'location': "right", 'shrink': 0.5}, zorder=1)

# Overlay hillshade
ax = show(hillshade, ax=ax, cmap='grey', clip_percent=10.0, skip=2, alpha=0.15, zorder=2)

# Overlay watershed
ax = show(watershed, ax=ax, color=(1.0, 1.0, 1.0, 0.3), edgecolor=(0.3, 0.3, 0.3, 0.5), linewidth=1.0, label='watershed', zorder=3)

# Overlay study area boundary
ax = show(study_area, ax=ax, color='black', linewidth=.5, label='study area', zorder=4)

#Overlay Streams 

ax = show(streams, ax=ax, color='dodgerblue', linewidth=0.75, label='streams', zorder=5)


# Overlay longest flow path
ax = show(LFP, ax=ax, color='red', linewidth=0.9, label='longest flowpath', zorder=5)

# Overlay outlet points
ax = show(outlet, ax=ax, marker='^', s=43, color=(1.0, 0.0, 0.0), linewidth=1.75, label='outlet', zorder=6)


# Set axes limits to DEM extent
ax.set_xlim([dem.configs.west, dem.configs.east])
ax.set_ylim([dem.configs.south, dem.configs.north])

# Add legend
ax.legend()

# save the map
fig.savefig(r'file_path\MEXAR_map.png', dpi=500, bbox_inches='tight')

# Show the map
plt.show()

